## Import Libraries and Load the Data

In [2]:
import numpy as np
import json

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import pickle
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [4]:
intents_file = open("intents.json").read()
print(intents_file)
intents = json.loads(intents_file)

{"intents": [
        {"tag": "greeting",
         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],
         "context": [""]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
         "context": [""]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
         "responses": ["Happy to help!", "Any time!", "My pleasure"],
         "context": [""]
        },
        {"tag": "noanswer",
         "patterns": [],
         "responses": ["Sorry, can't understand you", "Please give me more info", "Not sure I understand"],
         "context": [""]
        },
        {"tag": "optio

## Preprocessing the Data

In [18]:
words = []
classes = []
documents = []
ignore_letters = ['!','?',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        # add document in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [19]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes")
print(classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words")
print(words)

47 documents
9 classes
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words
["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [20]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

## Create Training and Testing Data

In [22]:
# create the training data
training = []

# create empty array for the output
output_empty = [0] * len(classes)

# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)

# create training and testing lists. X - patterns, Y - intents
X_train = list(training[:,0])
y_train = list(training[:,1])

## Training the Model

In [23]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [24]:
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               11264     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 585       
Total params: 20,105
Trainable params: 20,105
Non-trainable params: 0
_________________________________________________________________


In [26]:
#Training and saving the model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.3002 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1949 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1375 - accuracy: 0.1702
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0558 - accuracy: 0.2340
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9430 - accuracy: 0.3830
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7852 - accuracy: 0.5319
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7436 - accuracy: 0.5319
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5254 - accuracy: 0.5532
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5401 - accuracy: 0.5532
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2753 - accuracy: 0.7234
Epoch 11/

10/10 [==============================] - 0s 2ms/step - loss: 0.0245 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0376 - accuracy: 0.9787
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0414 - accuracy: 0.9787
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0293 - accuracy: 0.9787
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0390 - accuracy: 0.9787
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0487 - accuracy: 0.9787
Epoc